# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
!pip install chromadb

In [2]:
!pip install langchain_community

In [3]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
from langchain_openai import OpenAI, ChatOpenAI
llm = ChatOpenAI(model='gpt-4',temperature=0)

In [6]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "state_of_the_union.txt")

In [7]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [8]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [9]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [10]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [11]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 1051, which is longer than the specified 1000


## Create the Agent

In [12]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [13]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [14]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

/var/folders/d7/f99mzmr12dqbty5ptd50z0dc0000gn/T/ipykernel_9932/1834837320.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [15]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
This question is about the most recent state of the union address. I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: What did Biden say about Ketanji Brown Jackson in the state of the union address?
Observation: In the State of the Union address, Biden mentioned that he had nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court four days prior. He described her as one of the nation's top legal minds who will continue Justice Breyer’s legacy of excellence. He also mentioned that since her nomination, she has received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans.
Thought:I now know the final answer.
Final Answer: In the State of the Union address, Biden mentioned that he had nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United State

{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': "In the State of the Union address, Biden mentioned that he had nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court four days prior. He described her as one of the nation's top legal minds who will continue Justice Breyer’s legacy of excellence. He also mentioned that since her nomination, she has received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans."}

In [16]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
This question is about ruff, a python linter. I should use the Ruff QA System to answer this question.
Action: Ruff QA System
Action Input: Why should I use ruff over flake8?
Observation: Ruff can be used as a drop-in replacement for Flake8 under certain conditions, such as when used without or with a small number of plugins, alongside Black, and on Python 3 code. Ruff implements every rule in Flake8 under these conditions, including all of the F rules from Pyflakes and a subset of the E and W rules from pycodestyle. Additionally, Ruff re-implements some of the most popular Flake8 plugins and related code quality tools natively. Unlike Flake8, Ruff is capable of automatically fixing its own lint violations. Therefore, if these features align with your needs, you might prefer using Ruff over Flake8.
Thought:I now know the final answer.
Final Answer: Ruff can be used as a drop-in replacement for Flake8 under certain conditions, such as when used wi

{'input': 'Why use ruff over flake8?',
 'output': 'Ruff can be used as a drop-in replacement for Flake8 under certain conditions, such as when used without or with a small number of plugins, alongside Black, and on Python 3 code. Ruff implements every rule in Flake8 under these conditions, including all of the F rules from Pyflakes and a subset of the E and W rules from pycodestyle. Additionally, Ruff re-implements some of the most popular Flake8 plugins and related code quality tools natively. Unlike Flake8, Ruff is capable of automatically fixing its own lint violations. Therefore, if these features align with your needs, you might prefer using Ruff over Flake8.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [17]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [18]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [19]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
This question is about the most recent state of the union address. I should use the State of Union QA System to answer this.
Action: State of Union QA System
Action Input: What did Biden say about Ketanji Brown Jackson in the state of the union address?
Observation: In the State of the Union address, Biden mentioned that he had nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court four days prior. He described her as one of the nation's top legal minds who will continue Justice Breyer’s legacy of excellence. He also mentioned that since her nomination, she has received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': "In the State of the Union address, Biden mentioned that he had nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court four days prior. He described her as one of the nation's top legal minds who will continue Justice Breyer’s legacy of excellence. He also mentioned that since her nomination, she has received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans."}

In [20]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
This question is about ruff, a python linter. I should use the Ruff QA System to answer this question.
Action: Ruff QA System
Action Input: Why use ruff over flake8?
Observation: Ruff can be used as a drop-in replacement for Flake8 under certain conditions, such as when used without or with a small number of plugins, alongside Black, and on Python 3 code. Ruff implements every rule in Flake8 under these conditions, including all of the F rules from Pyflakes and a subset of the E and W rules from pycodestyle. 

Ruff also re-implements some of the most popular Flake8 plugins and related code quality tools natively. This can help minimize conflicts across plugins and allows any individual plugin to be toggled on or off with a single command. 

Unlike Flake8, Ruff is capable of automatically fixing its own lint violations, which can save developers time. 

However, it's important to note that Ruff does not support custom lint rules or third-party rul

{'input': 'Why use ruff over flake8?',
 'output': "Ruff can be used as a drop-in replacement for Flake8 under certain conditions, such as when used without or with a small number of plugins, alongside Black, and on Python 3 code. Ruff implements every rule in Flake8 under these conditions, including all of the F rules from Pyflakes and a subset of the E and W rules from pycodestyle. \n\nRuff also re-implements some of the most popular Flake8 plugins and related code quality tools natively. This can help minimize conflicts across plugins and allows any individual plugin to be toggled on or off with a single command. \n\nUnlike Flake8, Ruff is capable of automatically fixing its own lint violations, which can save developers time. \n\nHowever, it's important to note that Ruff does not support custom lint rules or third-party rules, which might be a limitation for some users."}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [21]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [22]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [23]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
The question is two-fold. First, I need to find out what tool ruff uses to run over Jupyter Notebooks. Then, I need to check if the president mentioned that tool in the state of the union. I'll start with the first part of the question.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation: The context does not provide information on what tool Ruff uses to run over Jupyter Notebooks.
Thought:I don't have the information about the tool ruff uses to run over Jupyter Notebooks. I can't proceed to the second part of the question without this information.
Final Answer: I'm sorry, but I don't have the information about the tool ruff uses to run over Jupyter Notebooks.

> Finished chain.


{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?',
 'output': "I'm sorry, but I don't have the information about the tool ruff uses to run over Jupyter Notebooks."}